In [17]:
import os
from collections import defaultdict
from gatenlp import Document
import json
import re

In [18]:
text_doc_dict = defaultdict(str)
for file in os.listdir("../data"):
    file_path = os.path.join("../data", file)
    if os.path.isfile(file_path):        
        with open(file_path, 'r') as fileReader:
          content = fileReader.read()
          text_doc_dict[file] = content

In [19]:
text_doc_dict.keys()

dict_keys(['DataGovernanceAct_Chapter_III.txt', 'DataAct_Chapter_IX.txt', 'DataGovernanceAct_Chapter_II.txt', 'GDPR_Chapter_X.txt', 'GDPR_Chapter_II.txt', 'GDPR_Chapter_XI.txt', 'GDPR_Chapter_III.txt', 'GDPR_intro.txt', 'DataAct_Chapter_III.txt', 'DataAct_Chapter_I.txt', 'AIAct_Chapter_X.txt', 'GDPR_Chapter_I.txt', 'AIAct_Chapter_XI.txt', 'DataGovernanceAct_Chapter_IX.txt', 'DataGovernanceAct_Chapter_VI.txt', 'DataAct_Chapter_IV.txt', 'AIAct_Chapter_XIII.txt', 'GDPR_Chapter_V.txt', 'AIAct_Chapter_VIII.txt', 'DataAct_Chapter_VII.txt', 'AIAct_Chapter_III.txt', 'AIAct_Chapter_IV.txt', 'GDPR_Chapter_VII.txt', 'DataGovernanceAct_Chapter_IV.txt', 'AIAct_Chapter_XII.txt', 'AIAct_Chapter_I.txt', 'DataGovernanceAct_Chapter_VII.txt', 'AIAct_Chapter_intro.txt', 'AIAct_Chapter_V.txt', 'DataGovernanceAct_Chapter_V.txt', 'DataAct_Chapter_II.txt', 'AIAct_Chapter_VII.txt', 'AIAct_Chapter_II.txt', 'GDPR_Chapter_IV.txt', 'DataGovernanceAct_Chapter_I.txt', 'AIAct_Chapter_VI.txt', 'DataAct-intro.txt', 'GD

In [ ]:
# Configuration for clustering
ENABLE_CLUSTERING = True  # Set to True to enable entity clustering
print(f'CLUSTERING_ENABLED={ENABLE_CLUSTERING}')

CLUSTERING_ENABLED=False


In [21]:
text_doc_dict.keys()

dict_keys(['DataGovernanceAct_Chapter_III.txt', 'DataAct_Chapter_IX.txt', 'DataGovernanceAct_Chapter_II.txt', 'GDPR_Chapter_X.txt', 'GDPR_Chapter_II.txt', 'GDPR_Chapter_XI.txt', 'GDPR_Chapter_III.txt', 'GDPR_intro.txt', 'DataAct_Chapter_III.txt', 'DataAct_Chapter_I.txt', 'AIAct_Chapter_X.txt', 'GDPR_Chapter_I.txt', 'AIAct_Chapter_XI.txt', 'DataGovernanceAct_Chapter_IX.txt', 'DataGovernanceAct_Chapter_VI.txt', 'DataAct_Chapter_IV.txt', 'AIAct_Chapter_XIII.txt', 'GDPR_Chapter_V.txt', 'AIAct_Chapter_VIII.txt', 'DataAct_Chapter_VII.txt', 'AIAct_Chapter_III.txt', 'AIAct_Chapter_IV.txt', 'GDPR_Chapter_VII.txt', 'DataGovernanceAct_Chapter_IV.txt', 'AIAct_Chapter_XII.txt', 'AIAct_Chapter_I.txt', 'DataGovernanceAct_Chapter_VII.txt', 'AIAct_Chapter_intro.txt', 'AIAct_Chapter_V.txt', 'DataGovernanceAct_Chapter_V.txt', 'DataAct_Chapter_II.txt', 'AIAct_Chapter_VII.txt', 'AIAct_Chapter_II.txt', 'GDPR_Chapter_IV.txt', 'DataGovernanceAct_Chapter_I.txt', 'AIAct_Chapter_VI.txt', 'DataAct-intro.txt', 'GD

In [22]:
gdocs = []
entities_path = "../entities_extraction_v1.2"

for key, value in text_doc_dict.items():
    
    base_doc = Document(value)
    base_doc.name = key.replace(".txt", "")
    ent_set = base_doc.annset("entities_")
    ent_json_path = os.path.join(entities_path, key.replace(".txt", ".json"))
    
    # Dictionary to store clusters by entity type, then by normalized text
    clusters_by_type = {}
    cluster_id = 1  # Progressive cluster ID counter
    
    with open(ent_json_path, "r") as json_file:
        ent_obj = json.load(json_file)
        for ent_type in ent_obj.keys():
            mapped_ent_type = ent_type.upper()
            
            if ENABLE_CLUSTERING:
                clusters_by_type[mapped_ent_type] = {}
            
            for entity_mention in ent_obj[ent_type]:
                # Escape special regex characters in the entity mention
                escaped_entity = re.escape(entity_mention)
                
                # Create a more flexible pattern for entities with special characters
                # Use word boundary only at the beginning if it starts with a word character
                # Use word boundary only at the end if it ends with a word character
                start_boundary = r'\b' if entity_mention[0].isalnum() else r'(?<!\w)'
                end_boundary = r'\b' if entity_mention[-1].isalnum() else r'(?!\w)'
                pattern = start_boundary + escaped_entity + end_boundary
                
                # Find all occurrences using regex
                counter = 0
                for match in re.finditer(pattern, value, re.IGNORECASE):
                    counter += 1
                    start = match.start()
                    end = match.end()
                    actual_text = value[start:end]  # Get the actual matched text (preserves case)
                    
                    # Add annotation with mapped entity type and get the actual annotation object with its ID
                    annotation = ent_set.add(start, end, mapped_ent_type, features={"text": actual_text})
                    actual_annotation_id = annotation.id  # Get the real GateNLP annotation ID
                    
                    if ENABLE_CLUSTERING:
                        # Use the actual matched text for clustering (normalized), only within same type
                        normalized_key = actual_text.lower().strip()
                        
                        if normalized_key not in clusters_by_type[mapped_ent_type]:
                            clusters_by_type[mapped_ent_type][normalized_key] = {
                                "id": cluster_id,  # Unique cluster ID
                                "title": actual_text,  # Use first occurrence as cluster title
                                "type": mapped_ent_type,
                                "nelements": 0,
                                "mentions": []
                            }
                            cluster_id += 1  # Increment for next cluster
                        
                        clusters_by_type[mapped_ent_type][normalized_key]["mentions"].append({
                            "id": actual_annotation_id,  # Use the real GateNLP annotation ID
                            "mention": actual_text
                        })
                        clusters_by_type[mapped_ent_type][normalized_key]["nelements"] = len(clusters_by_type[mapped_ent_type][normalized_key]["mentions"])
                
                print(f"Entity '{entity_mention}' in {key}: found {counter} matches")
    
    # Add clusters to document features in the correct format (only if clustering is enabled)
    if ENABLE_CLUSTERING and clusters_by_type:
        all_clusters = []
        for ent_type, type_clusters in clusters_by_type.items():
            all_clusters.extend(list(type_clusters.values()))
        
        if all_clusters:
            base_doc.features["clusters"] = {
                "entities_": all_clusters
            }
    
    gdocs.append(base_doc)

Entity 'Article 10' in DataGovernanceAct_Chapter_III.txt: found 5 matches
Entity 'Article 11' in DataGovernanceAct_Chapter_III.txt: found 1 matches
Entity 'Article 12' in DataGovernanceAct_Chapter_III.txt: found 3 matches
Entity 'Article 13' in DataGovernanceAct_Chapter_III.txt: found 2 matches
Entity 'Article 14' in DataGovernanceAct_Chapter_III.txt: found 1 matches
Entity 'Article 15' in DataGovernanceAct_Chapter_III.txt: found 1 matches
Entity 'Regulation (EU) 2016/679' in DataGovernanceAct_Chapter_III.txt: found 1 matches
Entity 'data intermediation services provider' in DataGovernanceAct_Chapter_III.txt: found 48 matches
Entity 'data holder' in DataGovernanceAct_Chapter_III.txt: found 5 matches
Entity 'data user' in DataGovernanceAct_Chapter_III.txt: found 4 matches
Entity 'data subjects' in DataGovernanceAct_Chapter_III.txt: found 15 matches
Entity 'legal representative' in DataGovernanceAct_Chapter_III.txt: found 11 matches
Entity 'competent authority for data intermediation ser

In [23]:
ent_obj.keys()

dict_keys(['REGULATION', 'ROLE', 'AUTHORITY', 'DATA.CAT', 'DATA.OP', 'RIGHT', 'RISK/IMPACT'])

In [24]:
for doc in gdocs:
  if not os.path.exists("../gatenlp_output"):
    os.mkdir("../gatenlp_output")
  file_path = os.path.join("../gatenlp_output", doc.name + '.json')
  with open(file_path, 'w') as fileWriter:
    json.dump(doc.to_dict(), fileWriter,  indent=4,)